In [10]:
import nest_asyncio
nest_asyncio.apply()
from pathlib import Path
from datetime import datetime
import re

import sys
sys.path.append("../")

from typing import List
from desci_sense.shared_functions.init import init_multi_chain_parser_config
from desci_sense.shared_functions.parsers.multi_chain_parser import MultiChainParser
from desci_sense.evaluation.utils import get_dataset, obj_to_json, obj_str_to_dict
from desci_sense.shared_functions.dataloaders import (
    scrape_post,
    convert_text_to_ref_post,
)
from desci_sense.shared_functions.configs import (
    OpenrouterAPIConfig,
    WandbConfig,
    LLMConfig,
    KeywordPParserChainConfig,
    RefTaggerChainConfig,
    TopicsPParserChainConfig,
    validate_env_var,
    MultiParserChainConfig,
    ParserChainType,
    PostProcessType,
) 

In [9]:
from desci_sense.shared_functions.interface import (
    RDFTriplet,
    isAConceptDefintion,
    KeywordConceptDefinition,
    ParserSupport,
    ParserResult,
    OntologyInterface,
    ZoteroItemTypeDefinition,
    )
from rdflib.namespace import RDF
from rdflib import URIRef, Literal, Graph

In [3]:
TEST_POST_TEXT_W_REF = """
I really liked this paper!
https://arxiv.org/abs/2402.04607
"""

In [4]:
multi_config = init_multi_chain_parser_config(llm_type='google/gemma-7b-it:free',
                                        post_process_type="combined")
multi_config.post_process_type = PostProcessType.COMBINED
mcp = MultiChainParser(multi_config)
res = mcp.process_text(TEST_POST_TEXT_W_REF)

2024-07-08 15:57:35.197 | INFO     | desci_sense.shared_functions.parsers.multi_chain_parser:__init__:76 - Initializing MultiChainParser. PostProcessType=combined
2024-07-08 15:57:35.199 | INFO     | desci_sense.shared_functions.parsers.multi_chain_parser:__init__:83 - Initializing post parsers...
2024-07-08 15:57:35.199 | INFO     | desci_sense.shared_functions.parsers.post_parser_chain:__init__:27 - Initializing parser chain 'multi_refs_tagger' 
2024-07-08 15:57:35.241 | INFO     | desci_sense.shared_functions.parsers.post_parser_chain:__init__:27 - Initializing parser chain 'topics' 
2024-07-08 15:57:35.268 | INFO     | desci_sense.shared_functions.parsers.post_parser_chain:__init__:27 - Initializing parser chain 'keywords' 
2024-07-08 15:57:35.296 | INFO     | desci_sense.shared_functions.parsers.post_parser_chain:__init__:27 - Initializing parser chain 'hashtags' 
2024-07-08 15:57:35.411 | WARNING  | desci_sense.shared_functions.web_extractors.citoid:before_retry:84 - Retry attemp

In [5]:
res.reference_urls

['https://arxiv.org/abs/2402.04607']

In [6]:
res.item_types

['preprint']

In [12]:
def convert_item_types_to_rdf_triplets(item_types: List[str], reference_urls: List[str]) -> List[RDFTriplet]:
    assert len(res.reference_urls) == len(res.item_types)
    triplets = [
        RDFTriplet(
            subject=URIRef(ref_url),
            predicate=URIRef(ZoteroItemTypeDefinition().uri),
            object=Literal(item_type),
        )
        for ref_url, item_type in zip(reference_urls, item_types)
    ]

    return triplets

In [13]:
convert_item_types_to_rdf_triplets(res.item_types, res.reference_urls)

ValidationError: 2 validation errors for RDFTriplet
subject.is-instance[Literal]
  Input should be an instance of Literal [type=is_instance_of, input_value='https://arxiv.org/abs/2402.04607', input_type=str]
    For further information visit https://errors.pydantic.dev/2.6/v/is_instance_of
subject.is-instance[URIRef]
  Input should be an instance of URIRef [type=is_instance_of, input_value='https://arxiv.org/abs/2402.04607', input_type=str]
    For further information visit https://errors.pydantic.dev/2.6/v/is_instance_of